In [1]:
! pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [idna]


In [2]:
! pip install numpy
! pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 18.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 2.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]


In [6]:
! pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 10.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]2m2/3 [pandas]


In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, TextVectorization, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import re
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

2025-06-12 19:14:58.824634: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-12 19:14:58.896244: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-12 19:14:58.944957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749734998.996813     796 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749734999.015262     796 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749734999.130956     796 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
with open('data/paragraphs.txt', 'r', encoding="utf-8") as file:
    para = file.read().lower()

In [3]:
para = re.sub(r'[^a-zA-Z\s]', '', para)
para = re.sub(r'\s+', ' ', para).strip()

In [4]:
dataset = tf.data.Dataset.from_tensor_slices([para])

E0000 00:00:1749735005.598497     796 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1749735005.603479     796 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
sequence_length = 100

In [6]:
vectorizer = TextVectorization(output_mode="int", split="whitespace", output_sequence_length=10000)
#fit the dataset to vectorizer
vectorizer.adapt(dataset)

#get vocabulary size
vocab_size = len(vectorizer.get_vocabulary())

# Converting text to sequences
sequences = vectorizer([para]).numpy()[0]

2025-06-12 19:15:05.934469: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [7]:
sequences

array([   22,   182,    56, ..., 31354,     5,     2])

In [8]:
X, Y = [], []
for i in range(len(sequences) - sequence_length):
    X.append(sequences[i:i+sequence_length])
    Y.append(sequences[i+sequence_length])

In [9]:
X = np.array(X)
Y = np.array(Y).flatten()
Y = to_categorical(Y, num_classes=vocab_size)

In [10]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, random_state=42, test_size=0.2)

In [11]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=256, input_length = sequence_length, mask_zero=True, embeddings_regularizer=l2(0.2)))
model.add(SimpleRNN(256, return_sequences=True, kernel_regularizer=l2(0.2)))
model.add(BatchNormalization()) 
model.add(Dropout(0.5))
model.add(SimpleRNN(128, kernel_regularizer=l2(0.2)))
model.add(BatchNormalization()) 
model.add(Dropout(0.5))
model.add(Dense(vocab_size, activation='softmax'))

/run/media/aadimprajapati/New Volume/fnclick/python/nlp/word-generation-rnn/env/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [12]:
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 256)       │    20,029,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 100, 256)       │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 100, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 128)            │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 78239)          │    10,092,831 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 90,910,943 (346.80 MB)

 Trainable params: 30,303,391 (115.60 MB)

 Non-trainable params: 768 (3.00 KB)

 Optimizer params: 60,606,784 (231.20 MB)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(xtrain, ytrain, epochs=50, batch_size=128, validation_data=(xtest, ytest), callbacks=[early_stop])

2025-06-12 19:15:25.112756: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2478611520 exceeds 10% of free system memory.


Epoch 1/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step - accuracy: 0.0000e+00 - loss: 3144.9712

2025-06-12 19:15:58.193332: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 619652880 exceeds 10% of free system memory.


62/62 ━━━━━━━━━━━━━━━━━━━━ 34s 482ms/step - accuracy: 0.0000e+00 - loss: 3140.6912 - val_accuracy: 5.0505e-04 - val_loss: 2356.6484
Epoch 2/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 25s 406ms/step - accuracy: 3.0323e-04 - loss: 2148.5881 - val_accuracy: 5.0505e-04 - val_loss: 1599.8998
Epoch 3/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 25s 399ms/step - accuracy: 0.0017 - loss: 1455.1245 - val_accuracy: 0.0020 - val_loss: 1074.2638
Epoch 4/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 25s 398ms/step - accuracy: 0.0030 - loss: 974.3444 - val_accuracy: 0.0071 - val_loss: 712.6936
Epoch 5/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 25s 401ms/step - accuracy: 0.0057 - loss: 644.6230 - val_accuracy: 0.0051 - val_loss: 467.4312
Epoch 6/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 25s 401ms/step - accuracy: 0.0084 - loss: 421.7772 - val_accuracy: 0.0096 - val_loss: 303.7783
Epoch 7/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 25s 401ms/step - accuracy: 0.0130 - loss: 273.6782 - val_accuracy: 0.0121 - val_loss: 196.5374
Epoch 8/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 25s 400ms/step - acc

In [16]:
vocab = vectorizer.get_vocabulary()
word_to_index = dict(zip(vocab, range(len(vocab))))
index_to_word = dict(enumerate(vocab))
10
def predict_next(seed_text, num_words=1):
    for _ in range(num_words):
        input_seq = vectorizer(seed_text).numpy()[:sequence_length]
        input_seq = np.expand_dims(input_seq, axis=0)
        prediction = model.predict(input_seq, verbose=0)
        predicted_index = np.argmax(prediction)
        predicted_word = index_to_word.get(predicted_index, "")
        if predicted_word in ("[UNK]", "[PAD]") or predicted_word.strip() == "":
            break
        seed_text += " " + predicted_word
    return seed_text

print(predict_next("My father", num_words=1))

My father injuries
